# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [2]:
# 查看目前上映那些電影，並擷取出其ID資訊
# url = 'https://movies.yahoo.com.tw/'
url = "https://movies.yahoo.com.tw/movie_intheaters.html"
response = requests.get(url)
response.encoding = 'utf-8'

result = BeautifulSoup(response.text, 'html5lib')
#result

In [3]:
on = result.find(class_ = "release_box").p.text
print("目前正在上映的電影：", on)

目前正在上映的電影： 共83筆，目前顯示1~10筆


In [4]:
html = result.find("select", attrs={'name':'movie_id'})

#正則表達式，取出option且屬性為data-name="電影名稱"的tag
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})  

for p in movie_item:
    #tag中的其他屬性可以直接以dict的方式取用
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

Movie: 蜘蛛人：無家日, ID: 11479
Movie: 攀登者, ID: 11948
Movie: 461個便當, ID: 11873
Movie: 死亡教室：母校, ID: 11825
Movie: 我從來不是幽默的女生, ID: 11824
Movie: 天菜特警, ID: 11821
Movie: 有你相伴的旅程, ID: 11757
Movie: 魔仿犯, ID: 11739
Movie: 北海浩劫, ID: 11711
Movie: 終極生存戰, ID: 11708
Movie: 藍宇, ID: 11695
Movie: ANNETTE：星夢戀歌, ID: 11635
Movie: 法蘭西特派週報, ID: 11633
Movie: 麥迪遜之橋, ID: 11617
Movie: 寶可夢：皮卡丘與可可的冒險, ID: 11607
Movie: 大紅狗克里弗, ID: 11416
Movie: 今晚來點史密斯, ID: 11805
Movie: 最後一個平安夜, ID: 11765
Movie: 仰望星空的少年, ID: 11764
Movie: 小教父全新4K完整導演版, ID: 11682
Movie: 哈利波特：神秘的魔法石, ID: 11677
Movie: 詭祭, ID: 11649
Movie: 爆裂戰警, ID: 11644
Movie: 青春特調蜂蜜檸檬蘇打, ID: 11631
Movie: 迷情蘇珊娜, ID: 11623
Movie: 魔法少女☆伊莉雅：LICHT無名的少女, ID: 11606
Movie: 獵魂者, ID: 11603
Movie: 美國女孩, ID: 11600
Movie: 生而為人, ID: 11589
Movie: 天才貓奴畫家, ID: 11538
Movie: 魔法滿屋, ID: 11736
Movie: 偶然與想像, ID: 11597
Movie: 電影之神, ID: 11568
Movie: 惡靈古堡首部曲：拉昆市, ID: 11558
Movie: 魔法阿媽, ID: 11552
Movie: Fate/Grand Order-終局特異點 冠位時間神殿所羅門-, ID: 11550
Movie: 阿查依蘭的呼喚, ID: 11378
Movie: 月老, ID: 11563
Movie:

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [5]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 11550

In [6]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
para = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
response = requests.get(url=url, params=para, headers=headers)
result = response.json()
#result
for data in result:
    print("上映地區：{}, 代號：{}".format(data["title"], data["area_id"]))


上映地區：台北市, 代號：28
上映地區：新北市, 代號：8
上映地區：桃園, 代號：16
上映地區：新竹, 代號：20
上映地區：台中, 代號：2
上映地區：台南, 代號：10
上映地區：高雄, 代號：17


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [7]:
# 指定放映地區
area_id = 28  #台北市

In [8]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
para = {
    'movie_id':str(movie_id), 
    'area_id':str(area_id)
    }
response = requests.get(url, params=para)
response.encoding = 'utf-8'

result = BeautifulSoup(response.text, 'html5lib')

#時間的位置
#   <label class="picker_label" for="date_0">
#   <p class="month">十二月</p>
#   <h3 class="day">15</h3>
#   <p class="week">今天</p>
#</label>



In [9]:
movie_date = result.find_all("label", attrs={'for':re.compile("date_[\d]")}) #正則表達式
# 列印播放日期
for date in movie_date:
    print("{}{}日".format(date.p.text, date.h3.text))

十二月15日
十二月16日
十二月17日
十二月18日
十二月19日


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [10]:
# 選定要觀看的日期
date = "2021-12-19"

In [11]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
para = {
    'movie_id':str(movie_id),
    'date':date,
    'area_id':str(area_id),
    'theater_id':'',
    'datetime':'',
    'movie_type_id':''
}

response = requests.get(url=url, params=para)
result = BeautifulSoup(response.json()["view"], "html5lib")

In [12]:
html =result.find_all("ul", attrs={'data-theater_name':re.compile(".*")})
for info in html:
  print("上映地點：", info.a.text)
  print("放映類型：", info.find(class_ = "tapR").text)
  print("場次時間：", info.find(class_ = "select").text, "\n")

上映地點： 京站威秀影城
放映類型： 數位
場次時間： 09:00 

